# Keras-Tensorflow Experiments

In [1]:
import pandas as pd
import numpy as np
import tensorflow as tf
import os
import sys
import glob
import time
from tensorflow import keras
from tensorflow import feature_column
from tensorflow.keras.models import Sequential, load_model
from tensorflow.keras.layers import Dense, Activation
from keras.utils.np_utils import to_categorical, normalize
from sklearn.model_selection import train_test_split, StratifiedShuffleSplit
from sklearn.preprocessing import LabelEncoder
from sklearn.utils import shuffle
from tensorflow.keras.callbacks import TensorBoard

Using TensorFlow backend.


## Download datasets
- if zip dataset files already exists; no download is done
- force unzip all the .zip files

In [2]:
%%bash
URL=https://iscxdownloads.cs.unb.ca/iscxdownloads/ISCX-URL-2016/
FILES=(ISCXURL2016.zip) 
for FILE in ${FILES[*]}; do
    if [ ! -f "$FILE" ]; then
        printf "downloading %s\n" $FILE
        curl -O $URL$FILE
        # unzip files
        echo 'unzipping ' $FILE
        unzip -o $FILE #overwrite exiting files/folders if exists
    fi
done

## Check CSV files inside FinalDataset folder

In [3]:
! ls FinalDataset

All_BestFirst.csv	      Malware_Infogain.csv
All_BestFirst_test.csv	      Malware_Infogain_test.csv
All.csv			      Phishing_BestFirst.csv
All.csv.pickle		      Phishing.csv
All_Infogain.csv	      Phishing_Infogain.csv
All_Infogain_test.csv	      Phishing_Infogain_test.csv
Defacement_BestFirst.csv      Spam_BestFirst.csv
Defacement.csv		      Spam_BestFirst_test.csv
Defacement_Infogain.csv       Spam.csv
Defacement_Infogain_test.csv  Spam_Infogain.csv
Malware_BestFirst.csv	      Spam_Infogain_test.csv
Malware.csv		      URL


## Analyze FinalDataset/All.csv file

In [4]:
df = pd.read_csv('FinalDataset/All.csv', low_memory=False)
resultPath = 'results_keras_tensorflow'
if not os.path.exists(resultPath):
    print('result path {} created.'.format(resultPath))
    os.mkdir(resultPath)

In [5]:
df.shape

(36707, 80)

In [6]:
df['argPathRatio'].astype('float')

0        0.076923
1        0.058824
2        0.060606
3        0.025974
4        0.040816
5        0.033898
6        0.046512
7        0.040000
8        0.045455
9        0.090909
10       0.043478
11       0.039216
12       0.095238
13       0.105263
14       0.080000
15       0.086957
16       0.038462
17       0.083333
18       0.017241
19       0.016949
20       0.020408
21       0.012579
22       0.014815
23       0.014085
24       0.012500
25       0.018182
26       0.050000
27       0.037037
28       0.039216
29       0.036364
           ...   
36677    0.792593
36678    0.763636
36679    0.794118
36680    0.015625
36681    0.057143
36682    0.085106
36683    0.662651
36684    0.100000
36685    0.407407
36686    0.036364
36687    0.578125
36688    0.129412
36689    0.712644
36690    0.785047
36691    0.086957
36692    0.040000
36693    0.068966
36694    0.742574
36695    0.015267
36696    0.071429
36697    0.985386
36698    0.817308
36699    0.275000
36700    0.052632
36701    0

In [7]:
df.columns

Index(['Querylength', 'domain_token_count', 'path_token_count',
       'avgdomaintokenlen', 'longdomaintokenlen', 'avgpathtokenlen', 'tld',
       'charcompvowels', 'charcompace', 'ldl_url', 'ldl_domain', 'ldl_path',
       'ldl_filename', 'ldl_getArg', 'dld_url', 'dld_domain', 'dld_path',
       'dld_filename', 'dld_getArg', 'urlLen', 'domainlength', 'pathLength',
       'subDirLen', 'fileNameLen', 'this.fileExtLen', 'ArgLen', 'pathurlRatio',
       'ArgUrlRatio', 'argDomanRatio', 'domainUrlRatio', 'pathDomainRatio',
       'argPathRatio', 'executable', 'isPortEighty', 'NumberofDotsinURL',
       'ISIpAddressInDomainName', 'CharacterContinuityRate',
       'LongestVariableValue', 'URL_DigitCount', 'host_DigitCount',
       'Directory_DigitCount', 'File_name_DigitCount', 'Extension_DigitCount',
       'Query_DigitCount', 'URL_Letter_Count', 'host_letter_count',
       'Directory_LetterCount', 'Filename_LetterCount',
       'Extension_LetterCount', 'Query_LetterCount', 'LongestPathToken

In [8]:
df.shape

(36707, 80)

In [9]:
df.head()

,Querylength,domain_token_count,path_token_count,avgdomaintokenlen,longdomaintokenlen,avgpathtokenlen,tld,charcompvowels,charcompace,ldl_url,...,SymbolCount_FileName,SymbolCount_Extension,SymbolCount_Afterpath,Entropy_URL,Entropy_Domain,Entropy_DirectoryName,Entropy_Filename,Entropy_Extension,Entropy_Afterpath,URL_Type_obf_Type
0,0,4,5,5.5,14,4.400000,4,8,3,0,...,1,0,-1,0.726298,0.784493,0.894886,0.850608,NaN,-1.0,Defacement
1,0,4,5,5.5,14,6.000000,4,12,4,0,...,0,0,-1,0.688635,0.784493,0.814725,0.859793,0.0,-1.0,Defacement
2,0,4,5,5.5,14,5.800000,4,12,5,0,...,0,0,-1,0.695049,0.784493,0.814725,0.801880,0.0,-1.0,Defacement
3,0,4,12,5.5,14,5.500000,4,32,16,0,...,0,0,-1,0.640130,0.784493,0.814725,0.663210,0.0,-1.0,Defacement
4,0,4,6,5.5,14,7.333334,4,18,11,0,...,0,0,-1,0.681307,0.784493,0.814725,0.804526,0.0,-1.0,Defacement


In [10]:
df.tail()

,Querylength,domain_token_count,path_token_count,avgdomaintokenlen,longdomaintokenlen,avgpathtokenlen,tld,charcompvowels,charcompace,ldl_url,...,SymbolCount_FileName,SymbolCount_Extension,SymbolCount_Afterpath,Entropy_URL,Entropy_Domain,Entropy_DirectoryName,Entropy_Filename,Entropy_Extension,Entropy_Afterpath,URL_Type_obf_Type
36702,29,4,14,5.750000,12,3.666667,4,20,24,3,...,3,2,7,0.690555,0.791265,0.777498,0.690227,0.656684,0.796205,spam
36703,0,4,13,3.750000,8,8.461538,4,24,23,0,...,16,15,-1,0.665492,0.820010,0.879588,0.674400,0.674671,-1.000000,spam
36704,58,3,27,6.666666,16,3.375000,3,41,34,20,...,8,7,9,0.656807,0.801139,0.684777,0.713622,0.717187,0.705245,spam
36705,35,3,13,4.333334,9,3.600000,3,15,13,7,...,9,8,3,0.725963,0.897617,0.871049,0.745932,0.758824,0.790772,spam
36706,40,3,25,6.666666,16,3.250000,3,35,31,19,...,7,6,7,0.674351,0.801139,0.697282,0.730563,0.731481,0.769238,spam


## Clean Data
- dropped samples with Infinity values
- dropped samples with NaN values

In [11]:
def loadData(csvFile):
    pickleDump = '{}.pickle'.format(csvFile)
    if os.path.exists(pickleDump):
        df = pd.read_pickle(pickleDump)
    else:
        df = pd.read_csv(csvFile, low_memory=False)
        # clean data
        # strip the whitspaces from column names
        df = df.rename(str.strip, axis='columns')
        #df.drop(columns=[], inplace=True)
        # drop missing values/NaN etc.
        #df.dropna(inplace=True)
        # drop Infinity rows and NaN string from each column
        for col in df.columns:
            indexNames = df[df[col]=='Infinity'].index
            if not indexNames.empty:
                print('deleting {} rows with Infinity in column {}'.format(len(indexNames), col))
                df.drop(indexNames, inplace=True)
            indexNames = df[df[col]=='NaN'].index
            if not indexNames.empty:
                print('deleting {} rows with NaN in column {}'.format(len(indexNames), col))
                df.drop(indexNames, inplace=True)
        
        df.to_pickle(pickleDump)
    
    return df

## Test loadData function

In [12]:
dataFile = 'FinalDataset/All.csv'
df = loadData(dataFile)

In [13]:
df.columns

Index(['Querylength', 'domain_token_count', 'path_token_count',
       'avgdomaintokenlen', 'longdomaintokenlen', 'avgpathtokenlen', 'tld',
       'charcompvowels', 'charcompace', 'ldl_url', 'ldl_domain', 'ldl_path',
       'ldl_filename', 'ldl_getArg', 'dld_url', 'dld_domain', 'dld_path',
       'dld_filename', 'dld_getArg', 'urlLen', 'domainlength', 'pathLength',
       'subDirLen', 'fileNameLen', 'this.fileExtLen', 'ArgLen', 'pathurlRatio',
       'ArgUrlRatio', 'argDomanRatio', 'domainUrlRatio', 'pathDomainRatio',
       'argPathRatio', 'executable', 'isPortEighty', 'NumberofDotsinURL',
       'ISIpAddressInDomainName', 'CharacterContinuityRate',
       'LongestVariableValue', 'URL_DigitCount', 'host_DigitCount',
       'Directory_DigitCount', 'File_name_DigitCount', 'Extension_DigitCount',
       'Query_DigitCount', 'URL_Letter_Count', 'host_letter_count',
       'Directory_LetterCount', 'Filename_LetterCount',
       'Extension_LetterCount', 'Query_LetterCount', 'LongestPathToken

In [14]:
df.shape

(36697, 80)

In [15]:
df['NumberRate_Extension'][:10]

0    1.0
1    NaN
2    NaN
3    NaN
4    NaN
5    NaN
6    NaN
7    NaN
8    NaN
9    1.0
Name: NumberRate_Extension, dtype: float64

In [16]:
df.shape

(36697, 80)

In [17]:
df.head()

,Querylength,domain_token_count,path_token_count,avgdomaintokenlen,longdomaintokenlen,avgpathtokenlen,tld,charcompvowels,charcompace,ldl_url,...,SymbolCount_FileName,SymbolCount_Extension,SymbolCount_Afterpath,Entropy_URL,Entropy_Domain,Entropy_DirectoryName,Entropy_Filename,Entropy_Extension,Entropy_Afterpath,URL_Type_obf_Type
0,0,4,5,5.5,14,4.400000,4,8,3,0,...,1,0,-1,0.726298,0.784493,0.894886,0.850608,NaN,-1.0,Defacement
1,0,4,5,5.5,14,6.000000,4,12,4,0,...,0,0,-1,0.688635,0.784493,0.814725,0.859793,0.0,-1.0,Defacement
2,0,4,5,5.5,14,5.800000,4,12,5,0,...,0,0,-1,0.695049,0.784493,0.814725,0.801880,0.0,-1.0,Defacement
3,0,4,12,5.5,14,5.500000,4,32,16,0,...,0,0,-1,0.640130,0.784493,0.814725,0.663210,0.0,-1.0,Defacement
4,0,4,6,5.5,14,7.333334,4,18,11,0,...,0,0,-1,0.681307,0.784493,0.814725,0.804526,0.0,-1.0,Defacement


## Experimenting with FinalDataset/All.csv

## Total samples for each type

In [18]:
# total samples
label = 'URL_Type_obf_Type'
lblTypes = set(df[label])
for lbl in lblTypes:
    print('| {} | {} |'.format(lbl, len(df[df[label] == lbl].index)))

| Defacement | 7930 |
| malware | 6711 |
| benign | 7781 |
| spam | 6698 |
| phishing | 7577 |


In [19]:
dataPath = 'FinalDataset'
dep_var = label
cat_names = []
cont_names = list(set(df.columns) - set(cat_names) - set([dep_var]))

In [20]:
cont_names

['SymbolCount_URL',
 'NumberRate_URL',
 'SymbolCount_Directoryname',
 'LongestVariableValue',
 'domain_token_count',
 'Filename_LetterCount',
 'Arguments_LongestWordLength',
 'sub-Directory_LongestWordLength',
 'domainlength',
 'urlLen',
 'pathLength',
 'Entropy_Afterpath',
 'URL_sensitiveWord',
 'charcompvowels',
 'Extension_DigitCount',
 'argPathRatio',
 'domainUrlRatio',
 'Query_DigitCount',
 'Directory_DigitCount',
 'tld',
 'dld_domain',
 'longdomaintokenlen',
 'delimeter_path',
 'ArgUrlRatio',
 'pathDomainRatio',
 'Domain_LongestWordLength',
 'URLQueries_variable',
 'Entropy_Filename',
 'NumberofDotsinURL',
 'dld_url',
 'argDomanRatio',
 'SymbolCount_FileName',
 'NumberRate_Domain',
 'SymbolCount_Afterpath',
 'delimeter_Count',
 'Entropy_DirectoryName',
 'fileNameLen',
 'LongestPathTokenLength',
 'delimeter_Domain',
 'avgdomaintokenlen',
 'File_name_DigitCount',
 'avgpathtokenlen',
 'host_letter_count',
 'charcompace',
 'Entropy_URL',
 'Path_LongestWordLength',
 'URL_DigitCount',


In [21]:
df['argPathRatio'] # was reading as object

0         0.07692308
1         0.05882353
2        0.060606062
3        0.025974026
4        0.040816326
5        0.033898305
6        0.046511628
7               0.04
8        0.045454547
9         0.09090909
10        0.04347826
11       0.039215688
12         0.0952381
13        0.10526316
14              0.08
15        0.08695652
16        0.03846154
17       0.083333336
18        0.01724138
19       0.016949153
20       0.020408163
21       0.012578616
22       0.014814815
23       0.014084507
24            0.0125
25       0.018181818
26              0.05
27       0.037037037
28       0.039215688
29       0.036363635
            ...     
36677      0.7925926
36678     0.76363635
36679      0.7941176
36680       0.015625
36681    0.057142857
36682     0.08510638
36683      0.6626506
36684            0.1
36685      0.4074074
36686    0.036363635
36687       0.578125
36688     0.12941177
36689      0.7126437
36690     0.78504676
36691     0.08695652
36692           0.04
36693     0.0

In [22]:
df.argPathRatio = df['argPathRatio'].astype('float')

In [23]:
for col in df.columns:
    print(df[col])

0          0
1          0
2          0
3          0
4          0
5          0
6          0
7          0
8          0
9          0
10         0
11         0
12         0
13         0
14         0
15         0
16         0
17         0
18         0
19         0
20         0
21         0
22         0
23         0
24         0
25         0
26         0
27         0
28         0
29         0
        ... 
36677     30
36678     41
36679     30
36680      0
36681      0
36682      0
36683     42
36684      0
36685      4
36686      0
36687     14
36688      2
36689     17
36690     41
36691      0
36692      0
36693      0
36694     40
36695      0
36696      0
36697    941
36698     51
36699      6
36700      0
36701      0
36702     29
36703      0
36704     58
36705     35
36706     40
Name: Querylength, Length: 36697, dtype: int64
0        4
1        4
2        4
3        4
4        4
5        4
6        4
7        4
8        4
9        4
10       4
11       4
12       4
13       4
14    

In [24]:
def baseline_model(feature_layer,inputDim=-1):
    global model_name
    model = tf.keras.Sequential([
        feature_layer,
        Dense(128, activation='relu', input_shape=(inputDim,)),
    #print(f"out_shape[1]:{out_shape[1]}")
        Dense(32, activation='relu'),
        Dense(5, activation='softmax')
    ]) #This is the output layer

    print('Categorical Cross-Entropy Loss Function')
    model_name += "_categorical"
    model.compile(optimizer='adam',
             loss='categorical_crossentropy',
             metrics=['accuracy'])
#         else:
#             model_name += "_binary"
#             print('Binary Cross-Entropy Loss Function')
#             model.compile(optimizer='adam',
#                     loss='binary_crossentropy',
#                     metrics=['accuracy'])
    return model

In [25]:
optimizer='adam'
epochs=10
batch_size=10

feature_columns = []

# numeric cols
for header in ['dld_getArg',
 'URLQueries_variable',
 'Directory_DigitCount',
 'URL_sensitiveWord',
 'spcharUrl',
 'URL_Letter_Count',
 'SymbolCount_Directoryname',
 'host_DigitCount']:
  feature_columns.append(feature_column.numeric_column(header))

feature_layer = tf.keras.layers.DenseFeatures(feature_columns)



def df_to_dataset(dataframe, shuffle=True, batch_size=32):
    dataframe=dataframe.copy()
    data_y=dataframe.pop(dep_var)
    encoder = LabelEncoder()
    encoder.fit(data_y)
    data_y = encoder.transform(data_y)
    dummy_y = to_categorical(data_y)
    dataframe = dataframe.copy()
    labels = dummy_y
    ds = tf.data.Dataset.from_tensor_slices((dict(dataframe), labels))
    if shuffle:
        ds = ds.shuffle(buffer_size=len(dataframe))
    ds = ds.batch(batch_size)
    return ds

train, test = train_test_split(df, test_size=0.2)
train, val = train_test_split(train, test_size=0.2)

batch_size = 32 # A small batch sized is used for demonstration purposes
train_ds = df_to_dataset(train, batch_size=batch_size)
val_ds = df_to_dataset(val, shuffle=False, batch_size=batch_size)
test_ds = df_to_dataset(test, shuffle=False, batch_size=batch_size)

    
#Creating data for analysis
time_gen = int(time.time())
global model_name
model_name = f"{dataFile}_{time_gen}"
#$ tensorboard --logdir=logs/
tensorboard = TensorBoard(log_dir='logs/{}'.format(model_name))

seed = 7
np.random.seed(seed)
cvscores = []
print('optimizer: {} epochs: {} batch_size: {}'.format(
    optimizer, epochs, batch_size))

#transform named labels into numerical values

#define 5-fold cross validation test harness
inputDim = 79
print('inputdim = ', inputDim)

#Separate out data
#X_train, X_test, y_train, y_test = train_test_split(data_x, dummy_y, test_size=0.2)
#num=0
#for train_index, test_index in sss.split(np.zeros(data_x.shape[0]), dummy_y):
    
    #X_train, X_test = data_x[train_index], data_x[test_index]
    #y_train, y_test = dummy_y[train_index], dummy_y[test_index]

    #create model
model = baseline_model(feature_layer, inputDim)

    #train
#print("Training " + dataFile + " on split " + str(num))
#model.fit(x=X_train, y=y_train, epochs=epochs, batch_size=batch_size, verbose=2, callbacks=[tensorboard], validation_data=(X_test, y_test))
model.fit(train_ds, validation_data=val_ds,epochs=epochs)
#save model
model.save('{}.model'.format(os.path.basename(dataPath)))

#num+=1


scores = model.evaluate(test_ds, verbose=1)
print(model.metrics_names)
acc, loss = scores[1]*100, scores[0]*100
print('Baseline: accuracy: {:.2f}%: loss: {:.2f}'.format(acc, loss))

resultFile = os.path.join(resultPath, 'All.csv')
with open('{}.result'.format(resultFile), 'a') as fout:
    fout.write('{} results...'.format(model_name))
    fout.write('\taccuracy: {:.2f} loss: {:.2f}\n'.format(acc, loss))

W0712 14:43:09.076110 140215374202496 deprecation.py:506] From /usr/lib/python3.7/site-packages/tensorflow/python/ops/init_ops.py:1251: calling VarianceScaling.__init__ (from tensorflow.python.ops.init_ops) with dtype is deprecated and will be removed in a future version.
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor


optimizer: adam epochs: 10 batch_size: 32
inputdim =  79
Categorical Cross-Entropy Loss Function
Epoch 1/10
734/734 [==============================] - 4s 5ms/step - loss: 1.4645 - acc: 0.4638 - val_loss: 1.2491 - val_acc: 0.5269
Epoch 2/10
734/734 [==============================] - 3s 4ms/step - loss: 1.1788 - acc: 0.5387 - val_loss: 1.2589 - val_acc: 0.4939
Epoch 3/10
734/734 [==============================] - 3s 4ms/step - loss: 1.1229 - acc: 0.5571 - val_loss: 1.1878 - val_acc: 0.5262
Epoch 4/10
734/734 [==============================] - 3s 4ms/step - loss: 1.0836 - acc: 0.5712 - val_loss: 1.0777 - val_acc: 0.5966
Epoch 5/10
734/734 [==============================] - 3s 4ms/step - loss: 1.0498 - acc: 0.5888 - val_loss: 1.0155 - val_acc: 0.6008
Epoch 6/10
734/734 [==============================] - 3s 4ms/step - loss: 1.0229 - acc: 0.5990 - val_loss: 1.0071 - val_acc: 0.6061
Epoch 7/10
734/734 [==============================] - 3s 4ms/step - loss: 1.0049 - acc: 0.6095 - val_loss: 0.98

In [26]:
preds, y, losses = model.get_preds(with_loss=True)
interp = ClassificationInterpretation(model, preds, y, losses)

AttributeError: 'Sequential' object has no attribute 'get_preds'

In [ ]:
interp.plot_confusion_matrix(slice_size=10)

In [ ]:
print(interp.confusion_matrix())

In [ ]:
interp.most_confused()